In [6]:
# imports
import os
import time
import datetime
import pandas as pd
import numpy as np
import math
import pandas_datareader as pdr
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def addcol_NR4(data, trend_periods=4, open_col='<OPEN>', high_col='<HIGH>', low_col='<LOW>',
                       close_col='<CLOSE>', drop_tr=True):
    for index, row in data.iterrows():
        prices = [row[open_col], row[high_col], row[low_col], row[close_col]]
        day_range = np.amax(prices) - np.amin(prices) 
        data.at[index,'day_range'] = day_range
      
        if index < 3:
            data.at[index, 'NR4'] = 0
            continue

        today = day_range
        tdm1 = data.at[index-1, 'day_range']
        tdm2 = data.at[index-2, 'day_range']
        tdm3 = data.at[index-3, 'day_range']
        ranges = [day_range, tdm1, tdm2, tdm3]

        if today == np.amin(ranges):
            data.at[index, 'NR4'] = 1 # yes a NR4 bar
        else:
            data.at[index, 'NR4'] = 0 # not a NR4 bar

    return data

In [3]:
def addcol_xmovavg(data, trend_periods=30, open_col='<OPEN>', high_col='<HIGH>', low_col='<LOW>',
                       close_col='<CLOSE>', drop_tab=True):
    col_name = str(trend_periods) + "mov_avg"  
    
    data[col_name] = data.iloc[:,4].rolling(window=trend_periods).mean()
    
    for x in range(trend_periods):
        data.at[x,col_name] = 0
    
    return data

In [4]:
def addcol_PL(data):
    addPL = data
    col_name = 'PL'
    addPL.at[0,col_name] = 0
    for index, row in data.iterrows():
        if row['trade'] == 'long':
            addPL.loc[index,'PL'] = row['exitprice'] - row['openprice']
        else: # short
            addPL.loc[index,'PL'] = row['openprice'] - row['exitprice']
    return addPL

In [5]:
def addcol_cumupl(data):
    addcumu = data
    col_name = 'cumulative'
    addcumu.at[0,col_name] = 0
        
    for index, row in addcumu.iterrows():
        if index ==0:
            addcumu.loc[index, col_name] = row['PL']
            continue
        addcumu.loc[index,col_name] = row['PL'] + addcumu.loc[index-1,col_name]
    
    return addcumu
    

In [6]:
def addcol_NDaysTrend(data, trend_periods=3, open_col='<OPEN>', high_col='<HIGH>', low_col='<LOW>',
                       close_col='<CLOSE>', drop_tab=True):
    
    col_name = str(trend_periods) + "days_trend"  
    strend = 'nodata'
    
    for index, row in data.iterrows():              
        prices = [row[open_col], row[high_col], row[low_col], row[close_col]]  
        
        if index < trend_periods:      
            data.at[index, col_name] = strend
            continue
                
        if index > trend_periods:          
            if prices[3] > data.at[index-1, 'Close'] and data.at[index-1, 'Close'] > data.at[index-2, 'Close']:
                strend = 'up'
            elif prices[3] < data.at[index-1, 'Close'] and data.at[index-1, 'Close'] < data.at[index-2, 'Close']:
                strend = 'down'
            else:
                strend = 'side'
                
        data.at[index, col_name] = strend
        
    return data

In [8]:
## RULES ################ DONT DELETE ###################################
# NR4 bar = 1, 3daytrend[0]?, if 3 days down, today.close > 20ma, LONG 
#    -> buy if next_day(high) > today close, exit at today(open) and PL at 5%

# NR4 bar = 1, 3daytrend[0]?, if 3 days up, today.close < 20ma, SHORT 
#    -> sell if next_day(low) > today close, exit at today(open) and PL at 5%
# need to get NR4, need to get 3days trend, need to get 30day MA
### RULES ################ DONT DELETE ###################################
def bts_NR4(symbol,data):    
    ### set open, high, low, close columns
    open_col = 'Open'
    high_col = 'High'
    low_col = 'Low'
    close_col = 'Close'
    
    ### set empty tradelogs
    columns = ['symbol','trade','opendate','openprice','closedate', 'exitprice']
    tradelogs = pd.DataFrame(columns=columns)
    tcounter = 0
    
    ### setup DF for the conditions to check and trades
    ma = 25
    ma_str = str(ma) + 'mov_avg'
    data = addcol_NR4(data, 4, 'Open','High','Low','Close')
    data= addcol_NDaysTrend(data, 3, 'Open','High','Low','Close')
    data = addcol_xmovavg(data, ma, 'Open','High','Low','Close')
    ### set to check 20mov_avg
    
    opentrade = False
    totrade = 'notrade'
    opendate =''
    openprice = 0
    exitdate =''
    exitprice = 0
    
    entryprice=0
    profitprice=0
    stopprice=0
    
    for index, row in data.iterrows():              
        prices = [row[open_col], row[high_col], row[low_col], row[close_col]]  
        
        #start checking for trades at position 21 - moving average (ma)
        if index < (ma+1):      
            continue
        
        ####step3 check if any open trades, if so, carry on until trade is closed by "continue"
        if opentrade:
            if tradelogs.loc[tcounter,'trade'] == 'short':
                if profitprice >= data.loc[index,'Low']:
                    tradelogs.loc[tcounter, 'closedate'] = data.loc[index,'Date']
                    tradelogs.loc[tcounter, 'exitprice'] = profitprice
                    opentrade = False
                    tcounter = tcounter + 1
                    totrade = 'notrade'
                elif stopprice <= data.loc[index,'High']:
                    tradelogs.loc[tcounter, 'closedate'] = data.loc[index,'Date']
                    tradelogs.loc[tcounter, 'exitprice'] = stopprice
                    opentrade = False
                    tcounter = tcounter + 1
                    totrade = 'notrade'
                else:
                    continue
            elif tradelogs.loc[tcounter,'trade'] == 'long':
                if profitprice <= data.loc[index,'High']:
                    tradelogs.loc[tcounter, 'closedate'] = data.loc[index,'Date']
                    tradelogs.loc[tcounter, 'exitprice'] = profitprice
                    opentrade = False
                    tcounter = tcounter + 1
                    totrade = 'notrade'
                elif stopprice >= data.loc[index,'Low']:
                    tradelogs.loc[tcounter, 'closedate'] = data.loc[index,'Date']
                    tradelogs.loc[tcounter, 'exitprice'] = stopprice
                    opentrade = False
                    tcounter = tcounter + 1
                    totrade = 'notrade'
                else:
                    continue
        
        ####step2 check if trade setup after triggered
        if (totrade == 'short' or totrade == 'long') and not opentrade:
            if totrade == 'short':
                #print('log short trade')
                if data.loc[index,'Low'] <= entryprice:
                    opentrade=True
                    tradelogs.loc[tcounter,'trade'] = totrade
                    tradelogs.loc[tcounter,'opendate'] = data.loc[index,'Date']
                    tradelogs.loc[tcounter,'openprice'] = entryprice 
                    tradelogs.loc[tcounter,'symbol'] = symbol
                    continue
                
            elif totrade == 'long': # long then TO-DO
                #print('log long trade')
                if data.loc[index,'High'] >= entryprice:
                    opentrade=True
                    tradelogs.loc[tcounter,'trade'] = totrade
                    tradelogs.loc[tcounter,'opendate'] = data.loc[index,'Date']
                    tradelogs.loc[tcounter,'openprice'] = entryprice
                    tradelogs.loc[tcounter,'symbol'] = symbol
                    continue ## added later, not sure if thisi s the right condition
        
        ####step1: check if setup is there and ready to be triggerd.
        if data.loc[index,'NR4']==1:
            #print('same day: ' + data.loc[index, '3days_trend'] + '  previous day: ' + data.loc[index-1, '3days_trend'])                      
            if data.loc[index, '3days_trend'] == 'up' or data.loc[index-1, '3days_trend'] == 'up':
                #print('TRADE DOWN')
                if data.loc[index, ma_str] >= data.loc[index, 'Close']: # downtrend
                    totrade = 'short'
                    entryprice = data.loc[index, 'Close']*0.995
                    profitprice = data.loc[index, 'Close']*0.94
                    stopprice = max(data.loc[index, 'Open'],data.loc[index, 'Close'])*1.005
                    
            elif data.loc[index, '3days_trend'] == 'down' or data.loc[index-1, '3days_trend'] == 'down':
                #print('TRADE UP')
                if data.loc[index, ma_str] <= data.loc[index,'Close']:
                    totrade = 'long'
                    entryprice = data.loc[index, 'Close']*1.005
                    profitprice = data.loc[index, 'Close']*1.06
                    stopprice = min(data.loc[index, 'Open'], data.loc[index, 'Close'])*0.995
            else:
                #print('dont trade')
                pass               


    return tradelogs
   
    

In [9]:
def get_symbolFromFilelist(flist):
    symbolfilelist = []
    for eitem in flist:
        symbol = eitem.split('.')
        symbolfilelist.append(symbol[0])
    return symbolfilelist

### provide folder path to this function
def get_folderFilelist(path):
    file_list = []
    file_list = os.listdir(path)  
    return file_list

In [10]:
def run_bts_allfiles(path):# need to implement for all test files.  then compare to this year with the drop.
    
    filelist = get_folderFilelist(path)
    symbollist = get_symbolFromFilelist(filelist)
    
    fullrun = pd.DataFrame()
    #### iterate through all files in 'path' variable folder
    for x in range(len(filelist)):
        stockpath = path+"\\"+filelist[x]
        stockdf = pd.read_csv(stockpath)
        #print(filelist[x].split('.')[0] +" : " + stockpath)
        stock_result = bts_NR4(filelist[x].split('.')[0], stockdf) 
        stock_result = addcol_PL(stock_result)
        fullrun = fullrun.append(stock_result, ignore_index=True)
        #result3 = result1.append(result2, ignore_index=True)
    return fullrun

In [14]:
#### run full test by call the run function
folder_path = "C:\\Users\\koon\\Desktop\\jupyter\\datafile"
fullresult = run_bts_allfiles(folder_path)

In [15]:
### set opendate from string to date field
fullresult['opendate'] = pd.to_datetime(fullresult['opendate'])
fullresult['closedate'] = pd.to_datetime(fullresult['closedate'])

In [16]:
### sorted by opend date
fullresult = fullresult.sort_values(by='opendate')

In [17]:
### reset index to opendate
fullresult = fullresult.set_index('opendate')

In [18]:
### drop incomplete trades
fullresult = fullresult.dropna()

In [ ]:
#### write a file with new file name:
ts = time.time()
st = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d_%H%M%S')
newfilename = 'bts_nr4v2_' + str(st)
fullresult.to_csv(newfilename+'.csv') 

In [35]:
fullresult.describe()

,PL
count,24454.000000
mean,0.157645
std,3.094766
min,-30.572696
25%,-0.718000
50%,-0.352600
75%,-0.105100
max,152.202604


In [36]:
fullresult.shape

(24454, 6)

In [37]:
fullresult.columns

Index(['symbol', 'trade', 'openprice', 'closedate', 'exitprice', 'PL'], dtype='object')

In [38]:
fullresult.head()

,symbol,trade,openprice,closedate,exitprice,PL
opendate,,,,,,
2016-02-11,A,short,35.6707,2016-02-12,35.5066,0.16410
2016-02-11,EA,short,58.6851,2016-02-12,58.4809,0.20415
2016-02-11,TPX,short,54.0385,2016-02-12,54.5816,-0.54310
2016-02-11,BZUN,short,5.3929,2016-02-12,5.0948,0.29810
2016-02-11,MPC,short,31.5117,2016-02-12,31.8786,-0.36695
